<a href="https://colab.research.google.com/github/Abhiie/ML_Lab/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
inputs = np.array([[73, 67, 43],
[91, 88, 64],
[87, 134, 58],
[102, 43, 37],
[69, 96, 70]], dtype='float32')
targets = np.array([[56],
[81],
[119],
[22],
[103]], dtype='float32')
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


In [ ]:
w=torch.randn(2,3,requires_grad=True)
b=torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[ 0.5580, -0.0700,  1.6597],
        [ 0.2807,  0.7859,  0.1630]], requires_grad=True)
tensor([-0.5519,  0.4738], requires_grad=True)


In [ ]:
def model(x):
  return x @ w.t() +b
preds=model(inputs)
print(preds)
print(targets)

tensor([[106.8568,  80.6273],
        [150.2836, 105.6058],
        [134.8726, 139.6544],
        [114.7613,  68.9300],
        [147.4055, 106.6945]], grad_fn=<AddBackward0>)
tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])


Loss function

In [ ]:
diff0 = preds - targets
torch.sum(diff0 * diff0) / diff0.numel()

tensor(2206.9727, grad_fn=<DivBackward0>)

In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
loss = mse(preds, targets)
print(loss)

tensor(2206.9727, grad_fn=<DivBackward0>)


 Compute gradients

In [ ]:
loss.backward()
print(w.grad)
print(b.grad)


tensor([[4784.7822, 3976.5920, 2816.4326],
        [2175.1262, 1791.1378, 1165.3446]])
tensor([54.6360, 24.1024])


Train the model using gradient descent

In [ ]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()
prediction = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(2206.9727, grad_fn=<DivBackward0>)


In [ ]:
for i in range(100):
    prediction = model(inputs)
    loss = mse(prediction, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(220.4753, grad_fn=<DivBackward0>)


Task 2

In [ ]:
import numpy as np
import torch
inputs1 = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
targets1 = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')
inputs = torch.from_numpy(inputs1)
targets = torch.from_numpy(targets1)
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [ ]:
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.]])
tensor([[ 56.,  70.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 81., 101.]])


torch.nn

In [ ]:
import torch.nn as nn
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1898,  0.3691,  0.4851],
        [-0.4642, -0.3644, -0.4176]], requires_grad=True)
Parameter containing:
tensor([0.4683, 0.3387], requires_grad=True)


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1898,  0.3691,  0.4851],
         [-0.4642, -0.3644, -0.4176]], requires_grad=True),
 Parameter containing:
 tensor([0.4683, 0.3387], requires_grad=True)]

In [ ]:
preds = model(inputs)
preds

tensor([[  32.2032,  -75.9197],
        [  46.7258, -100.6976],
        [  61.5551, -113.0973],
        [  14.9276,  -78.1300],
        [  56.7662,  -95.9064]], grad_fn=<AddmmBackward0>)

Loss function


In [ ]:
import torch.nn.functional as F
loss_fn = F.mse_loss
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(18920.6562, grad_fn=<MseLossBackward0>)


Train for multiple epochs

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 783.3048
Epoch [20/100], Loss: 232.2817
Epoch [30/100], Loss: 198.3366
Epoch [40/100], Loss: 177.1118
Epoch [50/100], Loss: 158.5409
Epoch [60/100], Loss: 142.1433
Epoch [70/100], Loss: 127.6569
Epoch [80/100], Loss: 114.8547
Epoch [90/100], Loss: 103.5364
Epoch [100/100], Loss: 93.5257


In [ ]:
prediction = model(inputs)
prediction

tensor([[ 59.3276,  74.0118],
        [ 83.3884,  98.3291],
        [112.5309, 132.3312],
        [ 33.1663,  57.7104],
        [ 97.0160, 103.0568]], grad_fn=<AddmmBackward0>)

In [ ]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[56.7628, 71.4071]], grad_fn=<AddmmBackward0>)